In [3]:
import numpy as np
from numpy import *
import matplotlib
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

from adventlib.common import *
# n, m, a = lines_as_matrix_nm(lines)
# a = lines_as_matrix(lines)
# a = lines_as_digit_matrix(lines)
# chunks = split_iter_by_start_indices(a, indices)
# chunks = split_iter_by_bools(a, bools)

from adventlib.point_2d import *
# from adventlib.point_3d import *

YEAR = 2022
DAY = int('14')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

In [4]:
lines, groups = linesplit("""
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
""".strip())

In [17]:
lines, groups = linesplit(raw_input, verbose=True)

142 lines
1 groups
>>>
497,29 -> 497,27 -> 497,29 -> 499,29 -> 499,25 -> 499,29 -> 501,29 -> 501,20 -> 501,29 -> 503,29 -> ...
477,77 -> 477,81 -> 473,81 -> 473,84 -> 484,84 -> 484,81 -> 483,81 -> 483,77
497,15 -> 497,16 -> 502,16
...


In [18]:
a = []

for line in lines:
  if m := re.findall(r'(-?\d+),(-?\d+)', line):
    ps = []
    for x, y in m:
      ps.append(Point(int(x), int(y)))
    a.append(ps)
a

[[[+497, +29],
  [+497, +27],
  [+497, +29],
  [+499, +29],
  [+499, +25],
  [+499, +29],
  [+501, +29],
  [+501, +20],
  [+501, +29],
  [+503, +29],
  [+503, +19],
  [+503, +29],
  [+505, +29],
  [+505, +25],
  [+505, +29],
  [+507, +29],
  [+507, +24],
  [+507, +29]],
 [[+477, +77],
  [+477, +81],
  [+473, +81],
  [+473, +84],
  [+484, +84],
  [+484, +81],
  [+483, +81],
  [+483, +77]],
 [[+497, +15], [+497, +16], [+502, +16]],
 [[+500, +157],
  [+500, +156],
  [+500, +157],
  [+502, +157],
  [+502, +156],
  [+502, +157],
  [+504, +157],
  [+504, +152],
  [+504, +157],
  [+506, +157],
  [+506, +155],
  [+506, +157],
  [+508, +157],
  [+508, +151],
  [+508, +157],
  [+510, +157],
  [+510, +153],
  [+510, +157]],
 [[+500, +157],
  [+500, +156],
  [+500, +157],
  [+502, +157],
  [+502, +156],
  [+502, +157],
  [+504, +157],
  [+504, +152],
  [+504, +157],
  [+506, +157],
  [+506, +155],
  [+506, +157],
  [+508, +157],
  [+508, +151],
  [+508, +157],
  [+510, +157],
  [+510, +153],
  [+5

In [19]:
points = set()

for ps in a:
  x = ps[0]
  points.add(x)
  for i in ps[1:]:
    while x != i:
      delta = i - x
      delta = Point(np.sign(delta.x), np.sign(delta.y))
      x = x + delta
      points.add(x)

In [20]:
FALL1 = Point(0, 1)
FALL2 = Point(-1, 1)
FALL3 = Point(1, 1)

In [21]:
taken = set(points)

sand_start = Point(500, 0)

count = 0

while True:
  x = sand_start
  
  for i in range(1000):
    if x + FALL1 not in taken:
      x = x + FALL1
    elif x + FALL2 not in taken:
      x = x + FALL2
    elif x + FALL3 not in taken:
      x = x + FALL3
    else:
      count += 1
      taken.add(x)
      break
  else:
    break

count

838

In [22]:
submit1(_)

Submitting 838...


True

170

In [24]:
taken = set(points)

maxy = max([i.y for i in points])

for i in range(-1000, +1001):
  taken.add(Point(i, maxy + 2))

sand_start = Point(500, 0)

count = 0

while True:
  x = sand_start
  
  if x in taken:
    break
  
  while True:
    if x + FALL1 not in taken:
      x = x + FALL1
    elif x + FALL2 not in taken:
      x = x + FALL2
    elif x + FALL3 not in taken:
      x = x + FALL3
    else:
      count += 1
      taken.add(x)
      break
  else:
    raise Exception()

count

27539

In [25]:
submit2(_)

Submitting 27539...


True

In [38]:
import dataclasses


@dataclasses.dataclass(eq=True, frozen=True)
class Point:
  x: int
  y: int

  def __add__(self, o: Point):
    return Point(self.x + o.x, self.y + o.y)

  def __sub__(self, o: Point):
    return Point(self.x - o.x, self.y - o.y)
  
  def __mul__(self, o: int):
    return Point(self.x * o, self.y * o)

  def __hash__(self):
    return hash((self.x, self.y))
  
  def __repr__(self):
    return f'[{self.x:+d}, {self.y:+d}]'
  
  def __matmul__(self, f):
    return Point(f(self.x), f(self.y))
  
  def __pos__(self):
    return self
  
  def __neg__(self):
    return self * -1
  
  def sign(self):
    return self @ np.sign
  
  def a(self):
    return np.array([self.x, self.y])

  def distance(a, b):
    return np.sqrt(np.sum((a - b).a() ** 2))
  
  def __abs__(self):
    return self.distance(ORIGIN)

  def mandistance(a, b):
    return np.sum(((a - b) @ np.abs).a())
  
  
  
  # def distance(self, o: Point):
  #   return np.sqrt()
a = Point(1, 1)
a.mandistance(ORIGIN)

2

In [43]:
a = []
a.append(a)

str(a)

'[[...]]'

In [45]:
d = {}
d['d'] = a

str(d)

"{'d': [[...]]}"